In [13]:
import sys
sys.path.append('..')
import pandas as pd
from PreprocessText import read_doc, preprocess_text
from sklearn.metrics.pairwise import cosine_similarity
from keyphrase_vectorizers import KeyphraseCountVectorizer
from transformers import BertModel, BertTokenizer
import torch
import re
import numpy as np
from typing import List
import nltk
from nomic import embed
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\johnt\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
def remove_useless_info(patent):
    positions_to_remove = {2, 3, 4}
    patent = [element for index, element in enumerate(patent) if index not in positions_to_remove]
    return patent

def remove_allcaps(patent):
    patent = [element for index, element in enumerate(patent) if not element.replace(" ", "").isupper()]
    return patent

def remove_starting_brackets(patent):
    for index, element in enumerate(patent):
        if element.startswith('[') and re.match(r'\[\d+\]', element):
            # Use regular expression to remove brackets and digits
            processed_string = re.sub(r'\[\d+\]', '', element)
            patent[index] = processed_string
    return patent

def process_paragraphs(patent: List[str]):
    patent = remove_useless_info(patent)
    patent = remove_allcaps(patent)
    patent = remove_starting_brackets(patent)
    return patent

In [3]:
path = r'C:\Users\johnt\Deep_Learning_Assignments\KeywordExtraction'

In [4]:
# Each patent is a big string, and all of the paragraphs are just separated by '\n'
patents, labels = read_doc(r'C:\Users\johnt\Downloads\20240106021625210-AMD (2).docx')

# Split each document into its paragraphs by splitting on '\n',
# So now each doc is a list of strings
# Then process all of these paragraphs according to the process_paragraphs function above
# Pop the last element from the list because it is an empty string paragraph
for i in range(len(patents)):
    patents[i] = patents[i].split('\n')
    patents[i] = process_paragraphs(patents[i])
    patents[i].pop()

In [14]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
doc_tokens = [tokenizer(paragraph, return_tensors="pt", add_special_tokens=False, max_length=512, truncation=True) for paragraph in patents[0]]

In [41]:
doc_tokens

[{'input_ids': tensor([[ 6110,  3384,  4796,  9099, 20483,  2869,  2007,  4359,  4796, 15772,
          17271,  4270]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])},
 {'input_ids': tensor([[ 4857,  4621,  4796, 15772, 14983,  2007,  4359, 17271,  4270,  2005,
           6110,  3384,  4796,  9099, 20483,  2869,  2003,  4719,  2011,  5716,
           1037,  9474,  6741,  2090,  1996,  4796, 15772,  6741,  1998,  3384,
           4796, 28688,  1010,  8558,  8161,  6911,  1012,  7861,  5092, 21341,
           2015,  2421,  5716,  1037,  9474,  6741,  1997, 16095,  8458,  3560,
           6351,  4820,  3384,  2482, 17062,  2229, 16922,  1999,  6693,  2013,
           1996,  3384,  4796, 28688,  2646,  1996,  4796, 15772,  6741,  2408,
           1996,  9474,  6741,  1012,  7861,  5092, 21341,  2015,  1997, 16134,
           2421,  9268,  1996,  2128, 25661,  4796,  1010, 12816,  2075,  1037,
           6

In [40]:
len(patents[0][4].split(" "))

48

In [42]:
total = 0
for i in range(len(patents)):
    doc_tokens = [tokenizer(paragraph, return_tensors="pt", add_special_tokens=False, max_length=512, truncation=True) for paragraph in patents[i]]
    for x in doc_tokens:
        total += len(x.input_ids[0])
        
print(total)
print(total/100)

991662
9916.62
